## Imports 

In [1]:
import os
from models.builers.retriever import Retriever

## Getting dataset

In [2]:
from data.dataloader import Data
import configparser

# load config.ini 
config = configparser.ConfigParser()
config.read('config.ini')
data_handler = Data(config)
dataset = "fiqa"
corpus, queries = data_handler.get_dataset(dataset)


Loading dataset from data/datasets\fiqa
data/datasets\fiqa\corpus.jsonl


In [3]:
relevants = data_handler.get_relevants(dataset)
# print first 5 relevants from the dictionary
print("First 5 relevants from the dictionary")
for i in list(relevants.keys())[:5]:
    print("  ", i, ":", relevants[i])

print("\nFirst query and document from corpus:")
print("  Corpus[0]:  ", corpus[0])
print("  Query[0]: ", queries[0])

First 5 relevants from the dictionary
   0 : [('18850', 1)]
   4 : [('196463', 1)]
   5 : [('69306', 1)]
   6 : [('560251', 1), ('188530', 1), ('564488', 1)]
   7 : [('411063', 1)]

First query and document from corpus:
  Corpus[0]:   {'_id': '3', 'title': '', 'text': "I'm not saying I don't like the idea of on-the-job training too, but you can't expect the company to do that. Training workers is not their job - they're building software. Perhaps educational systems in the U.S. (or their students) should worry a little about getting marketable skills in exchange for their massive investment in education, rather than getting out with thousands in student debt and then complaining that they aren't qualified to do anything.", 'metadata': {}}
  Query[0]:  {'_id': '0', 'text': 'What is considered a business expense on a business trip?', 'metadata': {}}


## Create or load models

In [4]:
documents = corpus[:10]
del corpus

In [5]:
# from models.model_loader_helpers import create_models

# models_to_create = {"TF-IDF": {},
#                     "BM25": {},
#                     "DPR": {},
#                     "Crossencoder": {"n":25},
#                     "KMeans": {"k":4},
#                     "CURE": {"k": 2, "n": 2, "shrinkage_fraction":0.2}}

# create_models(documents=documents, dataset_name=dataset, models=models_to_create, save=True)

from models.model_loader_helpers import load_models

models_to_load = {"TF-IDF": {},
                    "BM25": {},
                    "DPR": {}}
models = load_models("fiqa", models_to_load)

c:\Users\andre\OneDrive - Danmarks Tekniske Universitet\Masters\02807 Computational Tools for Data Science\venvCompTools\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Create Queries 

In [16]:
from data.query import Query

queries_ = data_handler.get_queries() # Get queries in the correct format
queries = []
for rel in relevants.items():
    id = rel[0]
    rels = [r[0] for r in rel[1]]
    query = queries_[id]
    queries.append(Query(text=query['text'], id=id, relevant_document_ids=rels))

# output an example
queries[0].GetQuery(), queries[0].GetNumberOfRelevantDocuments()

('What is considered a business expense on a business trip?', 1)

### Perform Experiment

In [115]:
def retrieveQueryAndGetScore(model: Retriever, query: Query, k: int):
    retrieved_documents = model.Lookup(query=query.GetQuery(), k=k)
    relevancies = []
    for document in retrieved_documents:
        if query.IsDocumentRelevant(document):
            relevancies.append(True)
        else:
            relevancies.append(False)
    return relevancies

In [116]:
def reciprocalRank(relevancies):
    for i, relevancy in enumerate(relevancies):
        if relevancy:
            return 1/(i+1)
    return 0

def meanReciprocalRank(scores):
    reciprocal_ranks = []
    for score in scores:
        reciprocal_ranks.append(reciprocalRank(score))

    return sum(reciprocal_ranks) / len(reciprocal_ranks)

def precision(relevancies):
    return sum([1 if relevancy else 0 for relevancy in relevancies]) / len(relevancies)

def recall(relevancies, query: Query):
    return sum([1 if relevancy else 0 for relevancy in relevancies]) / min(len(relevancies), query.GetNumberOfRelevantDocuments())

def calculate_metrics(results):
    """ Calculates precision, recall and accuracy based on the results of a query. True or false values are used to indicate whether a document is relevant or not.
    
    Args:
        results (list): A list of lists of booleans, where each list of booleans represents the retrieved documents for a query

    Returns:
        precision (float): Precision of the results
        recall (float): Recall of the results
        accuracy (float): Accuracy of the results
    """
    total_true_positives = 0  # Relevant documents correctly retrieved
    total_false_positives = 0 # Non-relevant documents incorrectly retrieved
    total_false_negatives = 0 # Relevant documents missed
    total_true_negatives = 0  # Non-relevant documents correctly not retrieved

    for query_results in results:
        true_positives = sum(query_results)
        false_positives = len(query_results) - true_positives
        # Assuming the length of the list is the total number of relevant documents for the query
        false_negatives = len(query_results) - true_positives  
        # True negatives can't be calculated without knowing the total number of non-relevant documents

        total_true_positives += true_positives
        total_false_positives += false_positives
        total_false_negatives += false_negatives

    # Calculating metrics
    precision = total_true_positives / (total_true_positives + total_false_positives) if (total_true_positives + total_false_positives) > 0 else 0
    recall = total_true_positives / (total_true_positives + total_false_negatives) if (total_true_positives + total_false_negatives) > 0 else 0
    accuracy = (total_true_positives + total_true_negatives) / (total_true_positives + total_false_positives + total_false_negatives + total_true_negatives) if (total_true_positives + total_false_positives + total_false_negatives + total_true_negatives) > 0 else 0

    mrr = meanReciprocalRank(results)

    return precision, recall, accuracy, mrr

In [117]:
import time

def timeFunction(function, **args):
    time_before = time.perf_counter()
    output = function(**args)
    time_after = time.perf_counter()
    return time_after - time_before, output

In [118]:
scores = {dataset:{}}
times = {dataset: {}}
for model_name in ["TF-IDF", "BM25"]:
# for model_name in list(models_to_load.keys()):
    scores[dataset][model_name] = list()
    times[dataset][model_name] = list()
    for query in queries:
        t_, s_ = timeFunction(retrieveQueryAndGetScore, **{"model": models[model_name], "query": query, "k": query.GetNumberOfRelevantDocuments()})
        scores[dataset][model_name].append(s_)
        times[dataset][model_name].append(t_)

In [119]:
print("Precision, Recall, Accuracy, MRR")
print("TF-IDF:", calculate_metrics(scores[dataset]["TF-IDF"]), "Avg. time:", sum(times[dataset]["TF-IDF"])/len(times[dataset]["TF-IDF"]))
print("BM25:", calculate_metrics(scores[dataset]["BM25"]), ", Avg. time:", sum(times[dataset]["BM25"])/len(times[dataset]["BM25"]))

Precision, Recall, Accuracy, MRR
TF-IDF: (0.0, 0.0, 0.0, 0.0) Avg. time: 0.00014021054545363852
BM25: (0.0002158273381294964, 0.0002158273381294964, 0.00010792531568154837, 0.0005454545454545455) , Avg. time: 5.471114545385221e-05


In [120]:

def create_latex_table(scores, times, caption="Experiment results.", label="tab:results-table"):
    """Prints a latex table from the scores and times dictionaries.
    
    Args:
        scores (dict): A dictionary containing the scores for each model and dataset
        times (dict): A dictionary containing the times for each model and dataset
        caption (str, optional): Caption of the table. Defaults to "Experiment results.".
        label (str, optional): Label of the table. Defaults to "tab:results-table".
    
    Returns:
        None
    """
    print("\\begin{table}[h]")
    print("\\begin{tabular}{ll|lllll}")
    print("\\textbf{Dataset} & \\textbf{Models} & \\textbf{Time} & \\textbf{Accuracy} & \\textbf{Precision} & \\textbf{Recall} & \\textbf{MRR} \\\\ \\hline")

    dataset_names = list(scores.keys())

    for dname in dataset_names:
        models_names = list(scores[dname].keys())
        # Loop through scores for all the models 
        for i, model_name in enumerate(models_names):
            precision, recall, accuracy, mrr = calculate_metrics(scores[dname][model_name])
            t_ = sum(times[dname][model_name])/len(times[dname][model_name])

            n_dec = 4
            # round numbers 
            precision = round(precision, n_dec)
            recall = round(recall, n_dec)
            accuracy = round(accuracy, n_dec)
            mrr = round(mrr, n_dec)
            t_ = round(t_, n_dec)

            if i==0:
                stri = f"\multirow{{{len(models_names)}}}{{*}}{{\\rotatebox[origin=c]{{90}}{{{dname}}}}} & {model_name} & {t_} s & {accuracy} & {precision} & {recall} & {mrr} \\\\"
            else:
                stri = f" & {model_name} & {t_} s & {accuracy} & {precision} & {recall} & {mrr} \\\\"
            print(stri)    
        
        print("\\hline")

    print("\\end{tabular}")
    print(f"\\caption{{{caption}}}")
    print(f"\\label{{{label}}}")
    print("\\end{table}")

In [121]:
create_latex_table(scores,times)

\begin{table}[h]
\begin{tabular}{ll|lllll}
\textbf{Dataset} & \textbf{Models} & \textbf{Time} & \textbf{Accuracy} & \textbf{Precision} & \textbf{Recall} & \textbf{MRR} \\ \hline
\multirow{2}{*}{\rotatebox[origin=c]{90}{fiqa}} & TF-IDF & 0.0001 s & 0.0 & 0.0 & 0.0 & 0.0 \\
 & BM25 & 0.0001 s & 0.0001 & 0.0002 & 0.0002 & 0.0005 \\
\hline
\end{tabular}
\caption{Experiment results.}
\label{tab:results-table}
\end{table}


## Run experiments on all datasets

!!! The following has not been tested yet

In [126]:
create_new_models = False # Set to True to create new models. False to load existing models

In [125]:
from data.dataloader import Data
import configparser

# load config.ini 
config = configparser.ConfigParser()
config.read('config.ini')
all_datasets = list(config["DATASETS"])
print("All datasets:", all_datasets)

# Store scores and retrieval times in dictionaries
scores = {}
times = {}

for dataset in all_datasets:
    print("Dataset:", dataset)

    # Get the dataset
    corpus, queries = data_handler.get_dataset(dataset)
    relevants = data_handler.get_relevants(dataset)
    documents = corpus
    del corpus
    

    # Define models with parameters
    models_to_create = {"TF-IDF": {},
                            "BM25": {},
                            "DPR": {},
                            "Crossencoder": {"n":25},
                            "KMeans": {"k":4},
                            "CURE": {"k": 2, "n": 2, "shrinkage_fraction":0.2}}
    
    # Either load or create models
    if create_new_models:
        from models.model_loader_helpers import create_models

        models = create_models(documents=documents, dataset_name=dataset, models=models_to_create, save=True)
    else:
        from models.model_loader_helpers import load_models
        
        models = load_models(dataset, models_to_create)

    # Get queries in the correct format
    queries_ = data_handler.get_queries() # Get queries in the correct format
    queries = []
    for rel in relevants.items():
        id = rel[0]
        rels = [r[0] for r in rel[1]]
        query = queries_[id]
        queries.append(Query(text=query['text'], id=id, relevant_document_ids=rels))

    # Now calculate the scores
    scores[dataset] = {}
    times[dataset] = {}
    for model_name in list(models_to_create.keys()):
        scores[dataset][model_name] = list()
        times[dataset][model_name] = list()
        for query in queries:
            t_, s_ = timeFunction(retrieveQueryAndGetScore, **{"model": models[model_name], "query": query, "k": query.GetNumberOfRelevantDocuments()})
            scores[dataset][model_name].append(s_)
            times[dataset][model_name].append(t_)
        print(f"{model_name}:", calculate_metrics(scores[dataset][model_name]), ", Avg. time:", sum(times[dataset][model_name])/len(times[dataset][model_name]))
    
    print("\n")

All datasets: ['quora', 'nfcorpus', 'msmarco', 'trec-covid', 'fiqa']


In [ ]:
# Create the latex table with the results
create_latex_table(scores,times)